In [2]:
import requests
import json
import csv

def write_csv(new_line):
    with open("common.csv", "a" ) as fout:
        writer=csv.writer(fout)
        writer.writerow((new_line))
        
pos=0.25

In [ ]:
#BTC-e
try:
    base_url="https://btc-e.nz/api/3/"
    pairs="btc_rur-btc_usd-dsh_btc-dsh_eth-dsh_ltc-dsh_usd-eth_btc-eth_ltc-eth_usd-ltc_btc-ltc_usd-nmc_btc-nmc_usd-nvc_btc-nvc_usd-ppc_btc-ppc_usd"

    ticker_url=base_url+"ticker/"+pairs
    depth_url=base_url+"depth/"+pairs

    html_ticker=requests.get(ticker_url).text
    html_depth=requests.get(depth_url).text

    ticker_data=json.loads(html_ticker)
    depth_data=json.loads(html_depth)
    
    for i in ticker_data:
        try:
            q=0
            w=[]
            e=[]
            for j in depth_data[i]["bids"]:
                q=q+j[1]
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)      
            r=0
            t=0
            for j in e:
                r=j[0]*j[1]+r
                t=j[1]+t    
            r=w[0][0]*(pos-t)+r
            bids=r/pos
        except:
            bids=ticker_data[i]["sell"]

        try:
            q=0
            w=[]
            e=[]
            for j in depth_data[i]["asks"]:
                q=q+j[1]
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)    
            r=0
            t=0
            for j in e:
                r=j[0]*j[1]+r
                t=j[1]+t    
            r=w[0][0]*(pos-t)+r
            asks=r/pos
        except:
            asks=ticker_data[i]["buy"]
    
    new_line=["btc-e", i, ticker_data[i]["buy"], ticker_data[i]["sell"], asks, bids]
    #new_line=[str(i), str(ticker_data[i]["buy"]), str(ticker_data[i]["sell"]), str(asks), str(bids)]
    print new_line                 
    
    write_csv(new_line)
except:
    pass

#bitfinex
try:    
    base_url="https://api.bitfinex.com/v1/"

    symbol_url=base_url+"symbols"
    html_symbol=requests.get(symbol_url).text
    symbol_data=json.loads(html_symbol)

    for i in symbol_data:
        ticker_url=base_url+"pubticker/"+i
        html_ticker=requests.get(ticker_url).text
        ticker_data=json.loads(html_ticker)     

        depth_url=base_url+"book/"+i
        html_depth=requests.get(depth_url).text
        depth_data=json.loads(html_depth)
    
    try:
        q=0
        w=[]
        e=[]
        for j in depth_data["bids"]:
            q=q+float(j["amount"])
            if q>pos:
                w.append(j)
            if q<pos:
                e.append(j)      
        r=0
        t=0
        for j in e:
            r=float(j["price"])*float(j["amount"])+r
            t=float(j["amount"])+t    
        r=float(w[0]["price"])*(pos-t)+r
        bids=r/pos
    except:
        bids=ticker_data["bid"]
    
    try:
        q=0
        w=[]
        e=[]
        for j in depth_data["asks"]:
            q=q+float(j["amount"])
            if q>pos:
                w.append(j)
            if q<pos:
                e.append(j)    
        r=0
        t=0
        for j in e:
            r=float(j["price"])*float(j["amount"])+r
            t=float(j["amount"])+t    
        r=float(w[0]["price"])*(pos-t)+r
        asks=r/pos     
    except:
        asks=ticker_data["ask"]
    
    new_line=["bitfinex", i, ticker_data["ask"], ticker_data["bid"], asks, bids]
    print new_line
    
    write_csv(new_line)
except:
    pass
    
# poloniex
try:
    base_url="https://poloniex.com/public?command=return"

    ticker_url=base_url+"Ticker"
    html_ticker=requests.get(ticker_url).text
    ticker_data=json.loads(html_ticker)

    for i in ticker_data:
        depth_url=base_url+"OrderBook&currencyPair="+i+"&depth=10"    
        html_depth=requests.get(depth_url).text
        depth_data=json.loads(html_depth)
        try:
            q=0
            w=[]
            e=[]
            for j in depth_data["bids"]:    
                q=q+j[1]
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)  
            r=0
            t=0
            for j in e:
                r=float(j[0])*j[1]+r
                t=j[1]+t    
            r=float(w[0][0])*(pos-t)+r
            bids=r/pos
        except:
            bids=ticker_data[i]["highestBid"]
    
        try:
            q=0
            w=[]
            e=[]
            for j in depth_data["asks"]:    
                q=q+j[1]
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)  
            r=0
            t=0
            for j in e:
                r=float(j[0])*j[1]+r
                t=j[1]+t    
            r=float(w[0][0])*(pos-t)+r
            asks=r/pos        
        except:
            asks=ticker_data[i]["lowestAsk"]
            
        #print i, ticker_data[i]["lowestAsk"], ticker_data[i]["highestBid"], asks, bids
        new_line=["poloniex", i, ticker_data[i]["lowestAsk"], ticker_data[i]["highestBid"], asks, bids]
        print new_line
        
        write_csv(new_line)
except:
    pass

# bittrex
try:
    base_url="https://bittrex.com/api/v1.1/public/"

    symbol_url=base_url+"getmarkets"
    html_symbol=requests.get(symbol_url).text
    symbol_data=json.loads(html_symbol)

    for i in symbol_data['result']:
        ticker_url=base_url+"getticker?market="+i['MarketName']
        html_ticker=requests.get(ticker_url).text
        ticker_data=json.loads(html_ticker)
    
        depth_url=base_url+"getorderbook?market="+i['MarketName']+"&type=both&depth=10"   
        html_depth=requests.get(depth_url).text
        depth_data=json.loads(html_depth)
    
        try:
            q=0
            w=[]
            e=[]
            for j in depth_data['result']['buy']:
                q=q+j['Quantity']
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j) 
            r=0
            t=0
            for j in e:
                r=j["Rate"]*j['Quantity']+r
                t=j['Quantity']+t    
            r=w[0]["Rate"]*(pos-t)+r
            bids=r/pos
        except:
            bids=ticker_data['result']["Bid"]
        
        try:
            q=0
            w=[]
            e=[]
            for j in depth_data['result']['sell']:
                q=q+j['Quantity']
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j) 
            r=0
            t=0
            for j in e:
                r=j["Rate"]*j['Quantity']+r
                t=j['Quantity']+t    
            r=w[0]["Rate"]*(pos-t)+r
            asks=r/pos
        except:
            asks=ticker_data['result']["Ask"]
        
        new_line = ['bittrex', i['MarketName'], ticker_data['result']["Ask"], ticker_data['result']["Bid"], asks, bids]
        print new_line
        
        write_csv(new_line)
except:
    pass
        
# kraken
try:
    base_url="https://api.kraken.com/0/public/"

    symbol_url=base_url+"AssetPairs"
    html_symbol=requests.get(symbol_url).text
    symbol_data=json.loads(html_symbol)

    pairs=[]
    z=0
    for i in symbol_data['result']:   
        pairs.append(symbol_data['result'][i]["altname"])    

    pairs=str(pairs).replace("[","").replace("]",'').replace("u",'').replace(" ","").replace("'","")

    ticker_url=base_url+"Ticker?pair="+pairs
    html_ticker=requests.get(ticker_url).text
    ticker_data=json.loads(html_ticker)

    for i in ticker_data['result']:
        depth_url=base_url+"Depth?pair="+symbol_data['result'][i]["altname"]+'&count=10'
        html_depth=requests.get(depth_url).text
        depth_data=json.loads(html_depth)
    
        try:
            q=0
            w=[]
            e=[]
            for j in depth_data['result'][i]['bids']:    
                q=q+float(j[1])
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)  
            r=0
            t=0
            for j in e:
                r=float(j[0])*j[1]+r
                t=j[1]+t    
            r=float(w[0][0])*(pos-t)+r
            bids=r/pos        
        except:
            bids=ticker_data['result'][i]['b'][0]        
        
        try:
            q=0
            w=[]
            e=[]
            for j in depth_data['result'][i]['asks']:    
                q=q+float(j[1])
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)  
            r=0
            t=0
            for j in e:
                r=float(j[0])*j[1]+r
                t=j[1]+t    
            r=float(w[0][0])*(pos-t)+r
            asks=r/pos        
        except:
            asks=ticker_data['result'][i]['a'][0]
        
        new_line=["kraken", symbol_data['result'][i]["altname"], ticker_data['result'][i]['a'][0], 
                  ticker_data['result'][i]['b'][0], asks, bids]
        print new_line
        
        write_csv(new_line)
except:
    pass
    
#hitbtc
try:
    base_url="https://api.hitbtc.com/api/1/public/"

    ticker_url=base_url+"ticker"
    html_ticker=requests.get(ticker_url).text
    ticker_data=json.loads(html_ticker)

    for i in ticker_data:
        depth_url=base_url+i+"/orderbook?format_price=number&format_amount=number"
        html_depth=requests.get(depth_url).text
        depth_data=json.loads(html_depth)
        
        try:
            q=0
            w=[]
            e=[]
            for j in depth_data["bids"]:
                q=q+j[1]
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)      
            r=0
            t=0
            for j in e:
                r=j[0]*j[1]+r
                t=j[1]+t    
            r=w[0][0]*(pos-t)+r
            bids=r/pos
        except:
            bids=ticker_data[i]["bid"]
    
        try:
            q=0
            w=[]
            e=[]
            for j in depth_data["asks"]:
                q=q+j[1]
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)      
            r=0
            t=0
            for j in e:
                r=j[0]*j[1]+r
                t=j[1]+t    
            r=w[0][0]*(pos-t)+r
            asks=r/pos
        except:
            asks=ticker_data[i]["ask"]
    
        new_line=["hitbtc", i, ticker_data[i]["ask"], ticker_data[i]["bid"], asks, bids]
        print new_line
        
        write_csv(new_line)
except:
    pass
    
#yobit
try:    
    base_url="https://yobit.net/api/3/"

    symbol_url=base_url+"info"
    html_symbol=requests.get(symbol_url).text
    symbol_data=json.loads(html_symbol)

    for i in symbol_data['pairs']:
        ticker_url=base_url+"ticker/"+i
        html_ticker=requests.get(ticker_url).text
        ticker_data=json.loads(html_ticker)
    
        depth_url=base_url+"depth/"+i+"?limit=10"
        html_depth=requests.get(depth_url).text
        depth_data=json.loads(html_depth)
    
        try:
            q=0
            w=[]
            e=[]
            for j in depth_data[i]["bids"]:
                q=q+j[1]
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)      
            r=0
            t=0
            for j in e:
                r=j[0]*j[1]+r
                t=j[1]+t    
            r=w[0][0]*(pos-t)+r
            bids=r/pos
        except:
            bids=ticker_data[i]['buy']

        try:
            q=0
            w=[]
            e=[]
            for j in depth_data[i]["asks"]:
                q=q+j[1]
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)    
            r=0
            t=0
            for j in e:
                r=j[0]*j[1]+r
                t=j[1]+t    
            r=w[0][0]*(pos-t)+r
            asks=r/pos
        except:
            asks=ticker_data[i]['sell']
    
        new_line=['yobit', i, ticker_data[i]['sell'], ticker_data[i]['buy'], asks, bids]
    
        print new_line
        write_csv(new_line)
except:
    pass
    
#cexio
try:
    base_url="https://c-cex.com/t/api_pub.html?a="

    ticker_url=base_url+"getmarketsummaries"
    html_ticker=requests.get(ticker_url).text
    ticker_data=json.loads(html_ticker)

    for i in ticker_data["result"]:
        try:
            depth_url=base_url+"getorderbook&market="+i['MarketName']+"&type=both&depth=10"
            html_depth=requests.get(depth_url).text
            depth_data=json.loads(html_depth)
    
            try:
                q=0
                w=[]
                e=[]
                for j in depth_data["result"]["sell"]:    
                    q=q+j['Quantity']
                    if q>pos:
                        w.append(j)
                    if q<pos:
                        e.append(j)  
                r=0
                t=0
                for j in e:
                    r=j['Rate']*j['Quantity']+r
                    t=j['Quantity']+t    
                r=w[0]['Rate']*(pos-t)+r
                bids=r/pos
            except:
                bids=i["Bid"]
        
            try:
                q=0
                w=[]
                e=[]
                for j in depth_data["result"]["buy"]:    
                    q=q+j['Quantity']
                    if q>pos:
                        w.append(j)
                    if q<pos:
                        e.append(j)  
                r=0
                t=0
                for j in e:
                    r=j['Rate']*j['Quantity']+r
                    t=j['Quantity']+t    
                r=w[0]['Rate']*(pos-t)+r
                asks=r/pos
            except:
                asks=i["Ask"]        
    
            new_line=["ccex", i['MarketName'], i["Bid"], i["Ask"], bids, asks]
            print new_line
    
            write_csv(new_line)
    
        except:
            continue
except:
    pass
    
#exmo
try:
    base_url="https://api.exmo.com/v1/"

    ticker_url=base_url+"ticker/"
    html_ticker=requests.get(ticker_url).text
    ticker_data=json.loads(html_ticker)

    for i in ticker_data:
        depth_url=base_url+"order_book/?pair="+i+"&limit=10"
        html_depth=requests.get(depth_url).text
        depth_data=json.loads(html_depth)
    
        try:
            q=0
            w=[]
            e=[]
            for j in depth_data[i]['bid']:    
                q=q+float(j[1])
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)  
            r=0
            t=0
            for j in e:
                r=float(j[0])*float(j[1])+r
                t=float(j[1])+t    
            r=float(w[0][0])*(pos-t)+r
            bids=r/pos
        except:
            bids=ticker_data[i]['buy_price']
        
        try:
            q=0
            w=[]
            e=[]
            for j in depth_data[i]['ask']:    
                q=q+float(j[1])
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)  
            r=0
            t=0
            for j in e:
                r=float(j[0])*float(j[1])+r
                t=float(j[1])+t    
            r=float(w[0][0])*(pos-t)+r
            asks=r/pos
        except:
            asks=ticker_data[i]['sell_price']
        
        new_line=["exmo", i, ticker_data[i]['buy_price'], ticker_data[i]['sell_price'], bids, asks]
    
        print new_line
        write_csv(new_line)
except:
    pass
    
#livecoin
try:
    base_url="https://api.livecoin.net/exchange/"

    ticker_url=base_url+"ticker"
    depth_url=base_url+"all/order_book"

    html_ticker=requests.get(ticker_url).text
    html_depth=requests.get(depth_url).text

    ticker_data=json.loads(html_ticker)
    depth_data=json.loads(html_depth)

    for i in ticker_data:
        try:
            q=0
            w=[]
            e=[]
            for j in depth_data[i["symbol"]]["bids"]:
                q=q+float(j[1])
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)             
            r=0
            t=0
            for j in e:
                r=float(j[0])*float(j[1])+r
                t=float(j[1])+t    
            r=float(w[0][0])*(pos-t)+r
            bids=r/pos     
        except:
            bids=i["best_bid"]
        
        try:
            q=0
            w=[]
            e=[]
            for j in depth_data[i["symbol"]]["asks"]:
                q=q+float(j[1])
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)              
            r=0
            t=0
            for j in e:
                r=float(j[0])*float(j[1])+r
                t=float(j[1])+t    
            r=float(w[0][0])*(pos-t)+r
            asks=r/pos      
        except:
            asks=i["best_ask"]        
        
        new_line = ["livecoin", i["symbol"], i["best_ask"], i["best_bid"], asks, bids]
        print new_line
        
        write_csv(new_line)
except:
    pass
    
# ccex
try:
    base_url="https://c-cex.com/t/api_pub.html?a="

    ticker_url=base_url+"getmarketsummaries"
    html_ticker=requests.get(ticker_url).text
    ticker_data=json.loads(html_ticker)

    for i in ticker_data["result"]:
        try:
            depth_url=base_url+"getorderbook&market="+i['MarketName']+"&type=both&depth=10"
            html_depth=requests.get(depth_url).text
            depth_data=json.loads(html_depth)
    
            try:
                q=0
                w=[]
                e=[]
                for j in depth_data["result"]["sell"]:    
                    q=q+j['Quantity']
                    if q>pos:
                        w.append(j)
                    if q<pos:
                        e.append(j)  
                r=0
                t=0
                for j in e:
                    r=j['Rate']*j['Quantity']+r
                    t=j['Quantity']+t    
                r=w[0]['Rate']*(pos-t)+r
                bids=r/pos
            except:
                bids=i["Bid"]
        
            try:
                q=0
                w=[]
                e=[]
                for j in depth_data["result"]["buy"]:    
                    q=q+j['Quantity']
                    if q>pos:
                        w.append(j)
                    if q<pos:
                        e.append(j)  
                r=0
                t=0
                for j in e:
                    r=j['Rate']*j['Quantity']+r
                    t=j['Quantity']+t    
                r=w[0]['Rate']*(pos-t)+r
                asks=r/pos
            except:
                asks=i["Ask"]        
    
            new_line=["ccex", i['MarketName'], i["Bid"], i["Ask"], bids, asks]
            print new_line
    
            write_csv(new_line)
    
        except:
            continue
except:
    pass

['btc-e', u'nvc_usd', 3.693, 3.62, 3.693, 3.62]
['bitfinex', u'iotbtc', u'0.00018206', u'0.00018199', 0.00018206, 0.00018199]
['poloniex', u'USDT_REP', u'27.17579326', u'26.50000079', 27.17579326, 26.50000079]
['poloniex', u'BTC_XVC', u'0.00016003', u'0.00016000', 0.00016003, 0.00016]
['poloniex', u'BTC_PINK', u'0.00000831', u'0.00000830', 8.31e-06, 8.3e-06]
['poloniex', u'BTC_SYS', u'0.00007503', u'0.00007500', 7.503e-05, 7.5e-05]
['poloniex', u'BTC_EMC2', u'0.00002444', u'0.00002441', 2.444e-05, 2.441e-05]
['poloniex', u'BTC_RADS', u'0.00221777', u'0.00220003', 0.00221777, 0.00220003]
['poloniex', u'BTC_SC', u'0.00000537', u'0.00000536', 5.37e-06, 5.36e-06]
['poloniex', u'BTC_MAID', u'0.00018657', u'0.00018482', 0.00018657, 0.00018483]
['poloniex', u'BTC_GNT', u'0.00020036', u'0.00020001', 0.00020036, 0.00019951]
['poloniex', u'BTC_BCN', u'0.00000117', u'0.00000116', 1.17e-06, 1.16e-06]
['poloniex', u'BTC_REP', u'0.01185961', u'0.01185957', 0.01185961, 0.01185957]
['poloniex', u'BTC_